In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
incomplete_n_samples = 2048
complete_n_samples = 4096

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3] 

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/', \
                             str(complete_n_samples), class_syn_id)

Give me the class type.
chair


In [6]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=30)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )

6778 files containing complete point clouds were found.


In [7]:
v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)        
test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id,
                                                       cmpl_pclouds, cmpl_model_names, n_threads=30,
                                                       n_samples=incomplete_n_samples)

6000 incomplete point clouds were loaded.


In [ ]:
# n_examples = len(test_recon)
n_examples = 200
# # pred_scores = np.zeros((n_examples, 2))
# bline_scores = np.zeros((n_examples, 2))

for i in xrange(n_examples):
#     gt = test_gt[i]
#     pred = test_recon[i]

    gt = fresh_from_mani(test_ids[i][:-6]).points
    feed = test_inc[i]

    bline_scores[i, 0] = accuracy_of_completion(feed, gt, thres=0.02, ret_dists=False)
    bline_scores[i, 1] = coverage_of_completion(gt, feed, thres=0.02, ret_dists=False)

#     pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
#     pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)

# np.savetxt

print np.average(bline_scores[:n_examples,0]), np.average(bline_scores[:n_examples,1])